In [1]:
from __future__ import absolute_import,division,print_function,unicode_literals
import os
import tensorflow as tf

import cProfile

### Basic of TF 2.0

In [2]:
# TF2.0 与传统编程相似 直接对实际的数据立即执行运算，操作会返回具体的值
tf.executing_eagerly()

True

In [3]:
tf.__version__

'2.0.0'

In [4]:
#  tf.matmul() 矩阵乘法
x = [[2.]]
m = tf.matmul(x,x)
print("x matmul x = {}".format(m))

x matmul x = [[4.]]


In [5]:
# 创建一个Tensor对象
a = tf.constant([[1,2],
                 [3,4]])
print(a)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


In [6]:
# python 中的广播机制
b = tf.add(a,1)
print(b)

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)


In [7]:
# a 和 b中对应元素想成 与tf.multiply()一样
print(a*b)

tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)


In [8]:
print(tf.multiply(a,b))

tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)


In [9]:
import numpy as np

In [10]:
# 把Tensor直接转换成ndarry格式
print(a.numpy())

[[1 2]
 [3 4]]


### Computing gradients

In [11]:
# tf中求梯度
w = tf.Variable([[1.0]])
with tf.GradientTape() as tape:
    loss = w * w
grad = tape.gradient(loss,w)
print(grad)

tf.Tensor([[2.]], shape=(1, 1), dtype=float32)


### Train a model

In [12]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train[:10000,:,:]
y_train = y_train[:10000]
x_test = x_test[:1000,:,:]
y_test = y_test[:1000]

In [13]:
x_train.shape

(10000, 28, 28)

In [14]:
# tf.newaxis 给Tensor 增加一个维度
x_train = tf.cast(x_train[...,tf.newaxis]/255,tf.float32)
x_test = tf.cast(x_test[...,tf.newaxis]/255,tf.float32)

In [15]:
x_train.shape

TensorShape([10000, 28, 28, 1])

In [16]:
# 将标签y转换成one-hot格式
y_train = tf.keras.utils.to_categorical(y_train,10)
y_test = tf.keras.utils.to_categorical(y_test,10)

In [17]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [18]:
# keras 建立使用Sequential建立Model
mnist_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,[3,3],activation='relu',input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(64,[3,3],activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10,activation='softmax')
])

In [19]:
mnist_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [20]:
# keras 使用model建立Model
inputs = tf.keras.Input(shape=(None,None,1),name="digits")
conv_1 = tf.keras.layers.Conv2D(16,[3,3],activation="relu")(inputs)
conv_2 = tf.keras.layers.Conv2D(16,[3,3],activation="relu")(conv_1)
ave_pool = tf.keras.layers.GlobalAveragePooling2D()(conv_2)
outputs = tf.keras.layers.Dense(10)(ave_pool)
mnist_model_2 = tf.keras.Model(inputs=inputs,outputs=outputs)

In [21]:
mnist_model_2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, None, None, 1)]   0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 16)    160       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 16)    2320      
_________________________________________________________________
global_average_pooling2d (Gl (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                170       
Total params: 2,650
Trainable params: 2,650
Non-trainable params: 0
_________________________________________________________________


### Two training methods

In [22]:
mnist_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
                   validation_split=0.1,shuffle=True,
                   loss = tf.keras.losses.categorical_crossentropy,
                   metrics=["accuracy"])

In [24]:
mnist_model.fit(x_train,y_train,batch_size=128,epochs=30)

Train on 10000 samples
Epoch 1/30
10000/10000 [==============================] - 18s 2ms/sample - loss: 2.1927 - accuracy: 0.2805
Epoch 2/30
10000/10000 [==============================] - 18s 2ms/sample - loss: 2.1447 - accuracy: 0.3345
Epoch 3/30
10000/10000 [==============================] - 18s 2ms/sample - loss: 2.0841 - accuracy: 0.3870
Epoch 4/30
10000/10000 [==============================] - 18s 2ms/sample - loss: 2.0045 - accuracy: 0.4343
Epoch 5/30
10000/10000 [==============================] - 18s 2ms/sample - loss: 1.8957 - accuracy: 0.4714
Epoch 6/30
10000/10000 [==============================] - 19s 2ms/sample - loss: 1.7765 - accuracy: 0.5019
Epoch 7/30
10000/10000 [==============================] - 18s 2ms/sample - loss: 1.6405 - accuracy: 0.5303
Epoch 8/30
10000/10000 [==============================] - 20s 2ms/sample - loss: 1.5050 - accuracy: 0.5681
Epoch 9/30
10000/10000 [==============================] - 24s 2ms/sample - loss: 1.3791 - accuracy: 0.5900
Epoch 10/30
10

In [25]:
mnist_model.evaluate(x_test,y_test)

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.44072437834739686, 0.873]

In [26]:
mnist_model_2.compile(optimizer=tf.keras.optimizers.Adam(),
                      loss = tf.keras.losses.SparseCategoricalCrossentropy(),
                      metrics=["accuracy"])

### Use TF 2.0

In [27]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train[:10000,:,:]
y_train = y_train[:10000]
x_test = x_test[:1000,:,:]
y_test = y_test[:1000]

In [28]:
dataset = tf.data.Dataset.from_tensor_slices(
    (tf.cast(x_train[...,tf.newaxis]/255,tf.float32),tf.cast(y_train,tf.int64))
)
dataset = dataset.shuffle(1000).batch(32)

In [29]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_history = []

In [30]:
for epoch in range(5):
    
    for (batch,(images,labels)) in enumerate(dataset):
        
        with tf.GradientTape() as tape:
            
            logits = mnist_model(images,training=True)
            loss_value = loss(labels,logits)
            
        grads = tape.gradient(loss_value,mnist_model.trainable_variables)
        optimizer.apply_gradients(zip(grads,mnist_model.trainable_variables))
        
    print("Epoch {} finishted".format(epoch))

Epoch 0 finishted
Epoch 1 finishted
Epoch 2 finishted
Epoch 3 finishted
Epoch 4 finishted
